# Hyperparameter exploration


In [ ]:
import os

from ultralytics import YOLO
from ray import tune

def tune_model(prefix, pt_model, data, space):
    name = prefix + '_' + pt_model.split('.')[0] + '_' + data.split('.')[0]
    
    task = pt_model.split('-')[1].split('.')[0]
    if task == 'seg':
        task = 'segment'
    elif task == 'obb':
        task = 'OBB'
    else:
        raise ValueError('Invalid task type. Must be seg or obb')
    
    data_path = os.path.join(os.getcwd(), data)

    model = YOLO(pt_model, task)
    result_grid = model.tune(data=data_path, use_ray=True, space=space, grace_period=4, batch=-1, epochs=20, name=name, device=0, degrees=180, gpu_per_trial=1)
   
    return result_grid

# models = {'yolov8s-seg.pt': [640,0.3,0],'yolov9c-seg.pt': [640,0.3,0], 'yolov8s-obb.pt': [800,0.2,0.25]}
models = ['yolov8s-seg.pt', 'yolov9c-seg.pt', 'yolov8s-obb.pt']

datasets = ['cgi.yaml', 'human.yaml', 'combined.yaml']

space = {
    'lr0': tune.uniform(1e-6, 2e-1),
    'momentum': tune.uniform(0.6, 0.98),
    'box': tune.uniform(0.02, 10),
    'cls': tune.uniform(0.2, 4.0),
    'dropout': tune.uniform(0, 0.5)
}

for pt in models:
    for dataset in datasets:
        result = tune_model('tuner', pt, dataset, space)


# Training models


In [ ]:
import os

from ultralytics import YOLO

def train_model(pt_model, data, prefix, img_size, dropout):
    name = prefix + '_' + pt_model.split('.')[0] + '_' + data.split('.')[0]
    
    task = pt_model.split('-')[1].split('.')[0]
    if task == 'seg':
        task = 'segment'
    elif task == 'obb':
        task = 'obb'
    else:
        raise ValueError('Invalid task type. Must be seg or obb')

    # check if folder exists
    if os.path.exists(os.path.abspath(f"./runs/{task}/{name}")):
        return

    epochs = 30
    patience = 15

    close_mosaic = 15
    batch_size = -1
    cache = False

    model = YOLO(pt_model)
    model.train(data=data, epochs=epochs, imgsz=img_size, patience=patience, batch=batch_size, cache=cache, name=name, dropout=dropout, device=0, degrees=180, close_mosaic=close_mosaic)
    return model

models = {
    'yolov8s-seg.pt': [480, 0.1],
    'yolov8s-obb.pt': [640, 0.2],
    'yolov9c-seg.pt': [480, 0.1]
}

datasets = ['cgi_sled.yaml', 'human_sled.yaml', 'combined_sled.yaml']

prefix = 'tiny'

for model in models:
    for data in datasets:
        train_model(model, data, prefix, models[model][0], models[model][1])
